# 常规赛简介
飞桨(PaddlePaddle)以百度多年的深度学习技术研究和业务应用为基础，集深度学习核心框架、基础模型库、端到端开发套件、工具组件和服务平台于一体，是中国首个开源开放、技术领先、功能完备的产业级深度学习平台。更多飞桨资讯，点击此处查看。

飞桨常规赛由百度飞桨于2019年发起，面向全球AI开发者，赛题范围广，涵盖领域多。常规赛旨在通过长期发布的经典比赛项目，为开发者提供学习锻炼机会，助力大家在飞将大赛中获得骄人成绩。

参赛选手需使用飞桨框架，基于特定赛题下的真实行业数据完成并提交任务。常规赛采取月度评比方式，为打破历史最高记录选手和当月有资格参与月度评奖的前10名选手提供飞桨特别礼包奖励。更多惊喜，更多收获，尽在飞桨常规赛。
比赛链接：https://aistudio.baidu.com/aistudio/competition/detail/20/0/introduction

# 导入对应模块

In [3]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
import random

# 数据准备


数据集	简介<br>
train.csv	训练集标签<br>
train_images.zip	训练集全部图片<br>
test_images.zip	测试集全部图片<br>

数据集链接：<br>
常规赛：中文场景文字识别-训练集：[https://aistudio.baidu.com/aistudio/datasetdetail/62842](https://aistudio.baidu.com/aistudio/datasetdetail/62842)<br>
常规赛：中文场景文字识别-测试集：[https://aistudio.baidu.com/aistudio/datasetdetail/62843](http://aistudio.baidu.com/aistudio/datasetdetail/62843)

In [ ]:
#解压数据
!unzip ./data/data62842/train_images.zip -d ./work/
!unzip ./data/data62843/test_images.zip -d ./work/
!cp ./data/data62842/train_label.csv -d ./work/

In [5]:
#把csv文件转换成txt
%cd /home/aistudio/
train_label_pd=pd.read_csv('./work/train_label.csv',encoding='GBK')
len(train_label_pd)
with open('./work/all.txt','w') as f:
    for i in range(len(train_label_pd)):
        name= train_label_pd['name'][i]
        label = train_label_pd['value'][i]
        f.write(name)
        f.write('\t')
        f.write(label)
        f.write('\n')

/home/aistudio


# 划分训练与验证数据
把训练集5%划分为验证集

In [7]:
with open('./work/all.txt','r') as f:
    lines=f.readlines()
number = len(lines)
print(number)
print(lines[0])
random.shuffle(lines)
print(lines[0])

with open('./work/train_list.txt','w') as f1:
    for i in range(int(len(lines)*0.95)):
        f1.write(lines[i])
print(int(len(lines)*0.95))
print(len(lines))
with open('./work/evl_list.txt','w') as f2:
    for i in range(int(len(lines)*0.95),len(lines)):
        f2.write(lines[i])


50000
0.jpg	拉拉

34281.jpg	专业精致儿童摄影

47500
50000


# clone paddleocr

In [6]:
# 下载代码
# !git clone https://gitee.com/paddlepaddle/PaddleOCR.git
# 安装运行所需要的whl包
os.chdir('/home/aistudio')
!pip install -U pip
!pip install -r ./PaddleOCR/requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 1.7MB 4.0MB/s eta 0:00:01     |██████████████████████▋         | 1.2MB 4.0MB/s eta 0:00:01
  Found existing installation: pip 19.2.3
    Uninstalling pip-19.2.3:
      Successfully uninstalled pip-19.2.3
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 1.1 MB 383 kB/s            
     |████████████████████████████████| 13.3 MB 5.1 MB/s             
     |████████████████████████████████| 948 kB 9.7 MB/s            
     |████████████████████████████████| 603 kB 9.6 MB/s            
     |████████████████████████████████| 298 kB 3.7 MB/s            
     |████████████████████████████████| 50 kB 9.8 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 55.7 MB 5.3 MB/s             
     |████████████████████████████████| 6.4 MB 2.8 MB/s            
     |████████████████████████████████| 57 kB

# 下载推理模型

In [ ]:
!wget https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_server_v2.0_rec_pre.tar
!tar -xf ch_ppocr_server_v2.0_rec_train.tar

--2021-12-26 15:33:51--  https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_server_v2.0_rec_pre.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 182.61.200.195, 182.61.200.229, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|182.61.200.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 490184704 (467M) [application/x-tar]
Saving to: ‘ch_ppocr_server_v2.0_rec_pre.tar’

ch_ppocr_server_v2. 100%[===================>] 467.48M  44.3MB/s    in 11s     

2021-12-26 15:34:02 (42.9 MB/s) - ‘ch_ppocr_server_v2.0_rec_pre.tar’ saved [490184704/490184704]



修改/PaddleOCR/configs/rec/ch_ppocr_v2.0/rec_chinese_common_train_v2.0.yml 配置文件开启训练


In [ ]:
#开启训练
os.chdir('/home/aistudio')
!python ./PaddleOCR/tools/train.py -c ./PaddleOCR/configs/rec/ch_ppocr_v2.0/rec_chinese_common_train_v2.0.yml 

[2022/01/14 09:12:08] root INFO: Architecture : 
[2022/01/14 09:12:08] root INFO:     Backbone : 
[2022/01/14 09:12:08] root INFO:         layers : 34
[2022/01/14 09:12:08] root INFO:         name : ResNet
[2022/01/14 09:12:08] root INFO:     Head : 
[2022/01/14 09:12:08] root INFO:         fc_decay : 4e-05
[2022/01/14 09:12:08] root INFO:         name : CTCHead
[2022/01/14 09:12:08] root INFO:     Neck : 
[2022/01/14 09:12:08] root INFO:         encoder_type : rnn
[2022/01/14 09:12:08] root INFO:         hidden_size : 256
[2022/01/14 09:12:08] root INFO:         name : SequenceEncoder
[2022/01/14 09:12:08] root INFO:     Transform : None
[2022/01/14 09:12:08] root INFO:     algorithm : CRNN
[2022/01/14 09:12:08] root INFO:     model_type : rec
[2022/01/14 09:12:08] root INFO: Eval : 
[2022/01/14 09:12:08] root INFO:     dataset : 
[2022/01/14 09:12:08] root INFO:         data_dir : /home/aistudio/work/train_images
[2022/01/14 09:12:08] root INFO:         label_file_list : ['/home/aist

In [ ]:
#evl
!python ./PaddleOCR/tools/eval.py -c ./PaddleOCR/configs/rec/ch_ppocr_v2.0/rec_chinese_common_train_v2.0.yml -o Global.checkpoints=output/rec_chinese_common_v2.0/best_accuracy

[2022/01/14 13:53:14] root INFO: Architecture : 
[2022/01/14 13:53:14] root INFO:     Backbone : 
[2022/01/14 13:53:14] root INFO:         layers : 34
[2022/01/14 13:53:14] root INFO:         name : ResNet
[2022/01/14 13:53:14] root INFO:     Head : 
[2022/01/14 13:53:14] root INFO:         fc_decay : 4e-05
[2022/01/14 13:53:14] root INFO:         name : CTCHead
[2022/01/14 13:53:14] root INFO:     Neck : 
[2022/01/14 13:53:14] root INFO:         encoder_type : rnn
[2022/01/14 13:53:14] root INFO:         hidden_size : 256
[2022/01/14 13:53:14] root INFO:         name : SequenceEncoder
[2022/01/14 13:53:14] root INFO:     Transform : None
[2022/01/14 13:53:14] root INFO:     algorithm : CRNN
[2022/01/14 13:53:14] root INFO:     model_type : rec
[2022/01/14 13:53:14] root INFO: Eval : 
[2022/01/14 13:53:14] root INFO:     dataset : 
[2022/01/14 13:53:14] root INFO:         data_dir : /home/aistudio/work/train_images
[2022/01/14 13:53:14] root INFO:         label_file_list : ['/home/aist

# 导出模型

In [ ]:
#导出模型
!python ./PaddleOCR/tools/export_model.py -c ./PaddleOCR/configs/rec/ch_ppocr_v2.0/rec_chinese_common_train_v2.0.yml  \
    -o Global.pretrained_model="./output/rec_chinese_common_v2.0/best_accuracy" \
    Global.save_inference_dir="./my_model"

W0114 13:53:43.639376 28522 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0114 13:53:43.643699 28522 device_context.cc:465] device: 0, cuDNN Version: 7.6.
[2022/01/14 13:53:48] root INFO: resume from /home/aistudio/output/rec_chinese_common_v2.0/latest
[2022/01/14 13:53:50] root INFO: inference model is saved to ./my_model/inference


# 预测结果
修改./tools/infer/predict_rec.py文件后台保存result.txt文件
![](https://ai-studio-static-online.cdn.bcebos.com/c27baf2e506e45c0a19c7eafdfec731924bd7b9f12104c40b49c423c2a4969cb)



In [8]:
#修改./tools/infer/predict_rec.py文件后台保存result.txt文件
import os
os.chdir('/home/aistudio/PaddleOCR/')
!python ./tools/infer/predict_rec.py --image_dir="../work/test_images" --rec_model_dir="/home/aistudio/my_model" --use_gpu=True

# 思路介绍
使用paddleocr预训练模型,能有效提升识别效果。
模型下载地址：https://gitee.com/paddlepaddle/PaddleOCR/blob/release/2.4/doc/doc_ch/models_list.md
![](https://ai-studio-static-online.cdn.bcebos.com/5ee4b5cf7b31452a92bad2ffe8a01f27d3e6c07d57c54fecb75cd23efb29c489)
本次使用的是：ch_ppocr_server_v2.0_rec预训练模型<br>
模型使用的是CRNN网络<br>
Backbone:使用的是resnet34<br>
Neck:使用的是rnn<br>
Head:使用的是ctchead<br>
参数设置：
epoch设置的是300轮。<br>
训练的时候发现前面几十轮就差不多是现在这个成绩，跑完了300轮分数还变低了。<br>
learning_rate: 0.001  应该还要设置的再小点。



# 总结
成绩为：74.89251	
还有很大的提升空间。
比如，把训练集里面，全角都改为半角，大写转小写，删除空字符等等。
然后再优化下参数等等。